# SFTT (Instruction Tuning) using Unsloth

In [ ]:
#install latest version of unsloth
#%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#%pip install --upgrade unsloth
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# %pip install --upgrade unsloth_zoo
# %pip install wandb

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset

import torch

# random state
SEED = 42

# reproducibility
## torch
torch.manual_seed(SEED)

## python
import random
random.seed(SEED)

## numpy
import numpy as np
np.random.seed(SEED)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🧠 Load Model

In [2]:
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_id = "meta-llama/Llama-3.2-1B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Init LoRA

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 1,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## Prepare Data

In [4]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset_cs = load_dataset("ctu-aic/cs_instruction_tuning_collection")
dataset_en = load_dataset("ctu-aic/en_instruction_tuning_collection")

In [5]:
dataset_en["train"][125]

{'original_id': 'alpaca-134',
 'conversations': [{'content': 'Classify the following data with three labels.\n\nfjsklfjdsklfjsklfjsklfjs',
   'role': 'user'},
  {'content': 'Invalid data. Please provide data in a clear and structured format for classification.',
   'role': 'assistant'}],
 'origin': 'bactrian-x-alpaca',
 'instruction_type': 'gpt-3.5',
 'instruction_translated': False,
 'output_type': 'gpt-3.5-turbo',
 'output_translated': False}

In [6]:
from datasets import concatenate_datasets

train = concatenate_datasets([dataset_cs["train"], dataset_en["train"]])
validation = concatenate_datasets([dataset_cs["validation"], dataset_en["validation"]])

In [7]:
from datasets import DatasetDict

train = train.shuffle(seed = SEED)
validation = validation.shuffle(seed = SEED)

train = train.select(range(20000))
validation = validation

dataset = DatasetDict({
    "train" : train,
    "validation" : validation,
})
dataset

DatasetDict({
    train: Dataset({
        features: ['original_id', 'conversations', 'origin', 'instruction_type', 'instruction_translated', 'output_type', 'output_translated'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['original_id', 'conversations', 'origin', 'instruction_type', 'instruction_translated', 'output_type', 'output_translated'],
        num_rows: 11033
    })
})

In [ ]:
# def standardize_ask_library(dataset):
#     """Standardize dataset to hugginface conversations format"""
#     # filter cs language only
#     dataset = dataset.filter(lambda x: x["language"] == "cs")

#     questions = dataset["question"]
#     answers = dataset["answer"]


#     conversations = []
#     for q, a in zip(questions, answers):
#         user = {"role": "user", "content": q}
#         assistant = {"role": "assistant", "content": a}
#         conversations.append([user, assistant])

#     dataset = dataset.add_column("conversations", conversations)

#     return dataset

In [8]:
#dataset = standardize_ask_library(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/11033 [00:00<?, ? examples/s]

In [9]:
dataset["train"][125]["conversations"]

[{'content': 'Determine the Sex of a Dwarf Hamster', 'role': 'user'},
 {'content': "It is important to know the sex of your hamsters if you have more than one and plan on keeping them in the same cage. You do not want your hamsters to end up breeding, especially if you're not equipped to care for a litter. You can examine your hamster's genitals directly to determine sex. If you can't determine sex this way, other factors such as size and scent glands can help. Be safe when sexing your hamster. You want to avoid accidents or injuries during the process.\n1. Flip your hamster over carefully. You want to pick your hamster up gently and then turn it on its back. Hamsters, especially young hamster, may resist this process. Go slowly and be gentle.\n\nCup your hand and allow your hamster to get into your hand. Once your hamster is seated on the palm of your hand, gently flip it over.\nYou can clasp your hand gently around the hamster's body and slowly flip it backwards. Without squeezing, h

In [10]:
dataset["train"][100]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nVytvořte vstup pro následující pokyn: napište příběh o ženě, která zkoumá jeskyni\n\nMary je zkušená speleologička, která je nadšená z průzkumu neobjevené jeskyně. Její přátelé ji varovali před nebezpečím jeskyně, včetně možnosti zřícení, ale ona je odhodlána jít dál.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nZačala svůj výzkum a brzy objevila křišťálové stěny, které se leskly světlem její svítilny. Její srdce se rozbušilo, jakmile spatřila kouzelný pohled na tento úžasný výtvor přírody.\n\nMary pokračovala dál a se zvědavostí pronikala hlouběji do jeskyně. Najednou cítila, že se jí zvedá žaludek, když se ocitla na úplně novém místě a uprostřed tmy najednou zahlédla malé světýlko.\n\nSnažila se přiblížit k této záři, ale když se dostala blíže, zjistila, že je to vrásčitá dlaně starého muž

In [25]:
#tokenize the texts to see how long the longest sequence is
tokenized = tokenizer(dataset["train"]["text"])

In [27]:
import numpy as np

lengths = [len(ids) for ids in tokenized["input_ids"]]

np.max(lengths), np.mean(lengths), np.std(lengths)

(np.int64(19151), np.float64(363.28375), np.float64(618.5232587671522))

In [28]:
dataset = dataset.shuffle(seed=SEED)

In [12]:
dataset.save_to_disk("data/it/init_mix_cs-en")

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11033 [00:00<?, ? examples/s]

## Train the model

In [29]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 256,
        gradient_accumulation_steps = 2,
        warmup_ratio = 0.01,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = SEED,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
        run_name="llama3.2-1B-it-init",
        eval_strategy = "steps",
        eval_steps = 50,
    ),
)

/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:578: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:592: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:606: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/20000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/11033 [00:00<?, ? examples/s]

In [30]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11033 [00:00<?, ? examples/s]

In [24]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKartavya Path byla nazývána ve jménu<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nzodpovědnosti a povinnosti. Tento termín se obvykle používá v hinduistické filozofii a kultuře a zdůrazňuje důležitost plnění svých povinností a zodpovědností vůči rodině, společnosti a celkově vůči životu. Je to filozofie, která nás vede k tomu, abychom neuplatňovali svá práva, ale abychom se zaměřili na to, co musíme udělat pro ostatní a pro svět jako celek.<|eot_id|>'

In [25]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                              \n\nzodpovědnosti a povinnosti. Tento termín se obvykle používá v hinduistické filozofii a kultuře a zdůrazňuje důležitost plnění svých povinností a zodpovědností vůči rodině, společnosti a celkově vůči životu. Je to filozofie, která nás vede k tomu, abychom neuplatňovali svá práva, ale abychom se zaměřili na to, co musíme udělat pro ostatní a pro svět jako celek.<|eot_id|>'

In [31]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.381 GB.
1.729 GB of memory reserved.


In [32]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 256 | Gradient Accumulation steps = 2
\        /    Total batch size = 512 | Total steps = 39
 "-____-"     Number of trainable parameters = 180,355,072
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlynatom. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [15]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4329.0353 seconds used for training.
72.15 minutes used for training.
Peak reserved memory = 19.932 GB.
Peak reserved memory for training = 17.297 GB.
Peak reserved memory % of max memory = 50.613 %.
Peak reserved memory for training % of max memory = 43.922 %.


## Inference

In [3]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Kdo je nejlepší fotbalista na světě?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, use_cache = True,
                         temperature = 1, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKdo je nejlepší fotbalista na světě?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTato otázka je vždy aktuální a způsobuje mnoho debat. Nejlepší fotbalista na světě je subjektivní a může se lišit podle osobních preferencí, statistik a individuálních výkonů. Nicméně, podle mnoha odborníků a fanoušků, jsou mezi nejlepšími fotbalisty na světě:\n\n1. **Lionel Messi**: Argentinský útočník, který hraje za Paris Saint-Germain, je často považován za jednoho z nejlepších fotbalistů všech dob. Jeho výjimečné technické dovednosti, rychlost, driblování a střelecké schopnosti ho činí úžasným hráčem.\n2. **Cristiano Ronaldo**: Portugalský útočník, který hraje za Al-Nassr, je další velkým jménem ve fotbalovém světě. Jeho neuvěřitelná síla, rychlost, střelecké schopnosti a vedení na hřišti ho činí jedním z

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nMarie Terezie je<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDobrý den, Marie Terezie se v českém jazyce nazývá Marie Terézie, je českou královskou královnou. Pozdějí se na dvě části: Marie - z řeckého word Marie (král, prince) a Terezie - z řeckého word Terésia (král) a to v době, kdy se královna pozdívala jako královna a pozdívala se jako královna. Dále se podepisovala jako Marie Terezie, kde se pozdívala jako královna a']